In [6]:
###get profile page
import sys
import os
import importlib
sys.path.insert(0,"twitterpackage/Chap02_03")
#print(sys.path)
#print(os.environ)

from twitter_client import get_twitter_client
import json
client = get_twitter_client()
profile = client.get_user(screen_name="Sysco")
#print(json.dumps(profile._json, indent =4))

In [3]:
!python ./twitterpackage/Chap02_03/twitter_get_user_timeline_daterange.py --username USFoods --startdate 2018-01-01

Length of Arguments Namespace(startdate='2018-01-01', username='USFoods')


In [7]:
###who is being mentioned
!python twitterpackage/Chap02_03/twitter_mention_frequency.py user_timeline_USFoods.jsonl

USFoods: 37
FoodFanatics: 20
BAP_Aquaculture: 11
ChsWineandFood: 11
NonGMOProject: 10
DeadmanCheffing: 8
Chefmathu: 6
usdaorganics: 6
Catersource_llc: 5
PrivatePropSC: 5
expandshare: 5
proj_angelheart: 5
Datassential: 5
WeRRestaurants: 4
MSCecolabel: 4
BSF_Corp: 4
FSRmag: 4
RnfrstAlliance: 4
ccapinc: 3
PapasBBQ: 3


In [ ]:
###who is being mentioned
!python twitterpackage/Chap02_03/twitter_mention_frequency.py user_timeline_USFoods.jsonl

In [9]:
#!python twitterpackage/Chap02_03/twitter_get_user.py FoodFanatics

More results available. Sleeping for 60 seconds to avoid rate limit


In [8]:
!python twitterpackage/Chap02_03/twitter_followers_stats.py FoodFanatics

Traceback (most recent call last):
  File "twitterpackage/Chap02_03/twitter_followers_stats.py", line 18, in <module>
    with open(followers_file) as f1, open(friends_file) as f2:
FileNotFoundError: [Errno 2] No such file or directory: 'users/FoodFanatics/followers.jsonl'


In [551]:
# !python twitterpackage/Chap02_03/twitter_get_user.py Sysco

In [2]:
# !python twitterpackage/Chap02_03/twitter_get_user.py USFoods

More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit


In [553]:
!python twitterpackage/Chap02_03/twitter_followers_stats.py Sysco

----- Timing -----
Initialize data: 0.5414795875549316
Set-based operations: 0.1800224781036377
Total time: 0.7215020656585693
----- Stats -----
Sysco has 12009 followers
Sysco has 452 friends
Sysco has 283 mutual friends
169 friends are not following Sysco back
11726 followers are not followed back by Sysco
Some mutual friends: TheIvyandJames, LaGarzaBurgers, SyscoRiverside, Bistro41Vnld, CompassTavernWo


In [554]:
!python twitterpackage/Chap02_03/twitter_followers_stats.py USFoods

Traceback (most recent call last):
  File "twitterpackage/Chap02_03/twitter_followers_stats.py", line 18, in <module>
    with open(followers_file) as f1, open(friends_file) as f2:
FileNotFoundError: [Errno 2] No such file or directory: 'users/USFoods/followers.jsonl'


In [555]:
!python twitterpackage/Chap02_03/twitter_influence.py USFoods Sysco

Traceback (most recent call last):
  File "twitterpackage/Chap02_03/twitter_influence.py", line 17, in <module>
    with open(followers_file1) as f1, open(followers_file2) as f2:
FileNotFoundError: [Errno 2] No such file or directory: 'users/USFoods/followers.jsonl'


In [620]:

html_template='''
<html>
    <head>
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css">
        <!-- *** <style>body{{ margin:0 0; background:whitesmoke; }}</style> *** --->
        <meta charset="utf-8" />
    </head>
    <body>
    <h3>Disaster Area Most Affected Customers</h3>
        <!-- *** Section 1 *** --->
        <h3>Price trends</h3>
        <h3>Table of Values<h3>
    </body>
</html>
'''





In [621]:
####Generate Mutual Friend Network
import os
import sys
import json
import time
import math
from tweepy import Cursor
from twitter_client import get_twitter_client
import sys
import json
from random import sample
import sys
import json
from operator import itemgetter
import networkx as nx
import matplotlib.pyplot as plt

screen_name = 'Sysco'
followers_file = 'users/{}/followers.jsonl'.format(screen_name)
friends_file = 'users/{}/friends.jsonl'.format(screen_name)
with open(followers_file) as f1, open(friends_file) as f2:
    t0 = time.time()
    followers = []
    friends = []
    for line in f1:
        profile = json.loads(line)
        followers.append(profile['screen_name'])
    for line in f2:
        profile = json.loads(line)
        friends.append(profile['screen_name'])
    t1 = time.time()
    mutual_friends = [user for user in friends if user in followers]
    followers_not_following = [user for user in followers if user not in friends]
    friends_not_following = [user for user in friends if user not in followers]
    t2 = time.time()
    print("Building Mutual Friend Network for {} Who Has {} Mutual Friends".format(screen_name, len(mutual_friends)))


G = nx.DiGraph()
for mutual in mutual_friends:
    #G.add_edge(screen_name,mutual)
    #G.add_edge(mutual,screen_name)
    G.add_node(mutual)

###Link the mutual friends network to Sysco
displaylabel = dict() 
for mutual in mutual_friends[:5]:
    screen_name = mutual
    followers_file = 'users/{}/followers.jsonl'.format(screen_name)
    try:
        with open(followers_file) as f1, open(friends_file) as f2:
            followers = []
            friends = []
            for line in f1:
                profile = json.loads(line)
                followers.append(profile['screen_name'])
            ###Add to the NetworkX Graph those that are connected
            followers_in_network = list(set(followers).intersection(mutual_friends))
            for u in followers_in_network:
                G.add_edge(u,screen_name)
            print("{} followers in network for {}".format(len(followers_in_network),screen_name))
            
            ###add the node display label
            displaylabel[screen_name] = '@' + screen_name + ": " +  profile['description']
            displaylabel[screen_name] = '<iframe src={}>'.format(html_template)
            #https://twitter.com/albrmar1
        
    except OSError:
        print("no file there for {}, removing node".format(screen_name))
        G.remove_node(screen_name)
    
    
    


Building Mutual Friend Network for Sysco Who Has 283 Mutual Friends
19 followers in network for SyscoKnox
18 followers in network for trycakenow
3 followers in network for SyscoLABSSL
0 followers in network for columbiadining
0 followers in network for warriorlounge
2 followers in network for chefmajor72
9 followers in network for SyscoKelowna
0 followers in network for factory_pastry
0 followers in network for boggsdj
5 followers in network for LibertyKitchen
14 followers in network for SyscoBoston
2 followers in network for AmyJoyDonutsInc
0 followers in network for cclfruit
11 followers in network for PearBureau
26 followers in network for USApears
5 followers in network for htpllc
5 followers in network for Sun_Maid
0 followers in network for gracecunning
2 followers in network for chefjustinHGC
4 followers in network for CaposRestaurant
9 followers in network for TeamFNV
0 followers in network for JellCraft
0 followers in network for matt_c_garrett
2 followers in network for Risou

42 followers in network for ProStart
11 followers in network for Foodable
29 followers in network for CertAngusBeef
29 followers in network for KidsLiveWell
3 followers in network for SoSonConsulting
11 followers in network for beardandbird
17 followers in network for wayneshurts
2 followers in network for Olympia_Grill
5 followers in network for matt7088
62 followers in network for NatlRestShow
1 followers in network for cathy_henry63
1 followers in network for GelatoFresco
0 followers in network for Bistro41Vnld
3 followers in network for foodie_photo
0 followers in network for EmpireForceNYC
2 followers in network for FatBoyzPizza
0 followers in network for CornerBarMpls
0 followers in network for IntlSportsCamp
0 followers in network for ColombosReading
14 followers in network for LeahMcGrathRD
11 followers in network for FreshPointTN
25 followers in network for CaWalnuts
0 followers in network for cataliascafe
0 followers in network for NSS_Foodservice
0 followers in network for k

In [622]:
import matplotlib as mpl
import visJS2jupyter.visJS_module

#Create network layout for visualizations
pos = nx.spring_layout(G , scale= 1)

#plt.axis("off")
#nx.draw_networkx(G, pos = pos, with_labels = False, arrows=False, node_size = 35)

nodes = G.nodes()
edges = list(G.edges())
nodes_dict = [{"id":n,
               "color":'black',
              "title": displaylabel[n],
              "displaylabel":n,
              "x":pos[n][0]*1000,
              "y":pos[n][1]*1000} for n in nodes
              ]
node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

edges_dict = [{"source":node_map[edges[i][0]], 
               "target":node_map[edges[i][1]], 
              "color":"gray"
              } for i in range(len(edges))]

# set some network-wide styles
NodeGraph = visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                          node_label_field='displaylabel',
                          node_size_multiplier=10,
                          node_size_transform = '',
                          node_color_highlight_border='red',
                          node_color_highlight_background='#D3918B',
                          node_color_hover_border='blue',
                          node_color_hover_background='#8BADD3',
                          node_font_size=25,
                          edge_arrow_to=False,
                          physics_enabled=True,
                          edge_color_highlight='#8A324E',
                          edge_color_hover='#8BADD3',
                          edge_label_field='label',
                          edge_title_field='title',
                          edge_width=3,
                          max_velocity=15,
                          min_velocity=1,
                          graph_id = 1)

NodeGraph

In [590]:
bt = nx.betweenness_centrality(G)
nx.set_node_attributes(G, name='betweenness', values=bt)
nx.set_node_attributes(G, name='size', values=5)
nx.set_node_attributes(G, name='color', values="black")

###number of influencers to identify
top = 5
max_nodes =  sorted(bt.items(), key = lambda v: -v[1])[:top]

for max_key, max_val in max_nodes:
    G.node[max_key]['size'] = 15
    G.node[max_key]['color'] = 'yellow'

color = nx.get_node_attributes(G,'color')
size = nx.get_node_attributes(G,'size')


In [591]:
nodes = G.nodes()
edges = list(G.edges())
nodes_dict = [{"id":n,
               "color":color[n],
              "title": displaylabel[n],
              "displaylabel":n,
              "degree":size[n],
              "x":pos[n][0]*1000,
              "y":pos[n][1]*1000} for n in nodes
              ]
node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

edges_dict = [{"source":node_map[edges[i][0]], 
               "target":node_map[edges[i][1]], 
              "color":"gray"
              } for i in range(len(edges))]

# set some network-wide styles
NodeGraphInfluencer = visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                          node_label_field='displaylabel',
                          node_size_multiplier=10,
                          node_color_highlight_border='red',
                          node_color_highlight_background='#D3918B',
                          node_color_hover_border='blue',
                          node_color_hover_background='#8BADD3',
                          node_font_size=25,
                          edge_arrow_to=False,
                          physics_enabled=True,
                          edge_color_highlight='#8A324E',
                          edge_color_hover='#8BADD3',
                          edge_label_field='label',
                          edge_title_field='title',
                          edge_width=3,
                          max_velocity=15,
                          min_velocity=1,
                          graph_id = 2)

NodeGraphInfluencer

In [593]:
import community

Gc = G.to_undirected()
parts = community.best_partition(Gc)
values = [parts.get(node) for node in Gc.nodes()]

nx.set_node_attributes(Gc, name='community', values=parts)
#nx.set_node_attributes(Gc, name='size', values=5)

color = visJS2jupyter.visJS_module.return_node_to_color(Gc,field_to_map='community',cmap=plt.get_cmap("rainbow"))
size = nx.get_node_attributes(Gc,'size')

nodes = Gc.nodes()
edges = list(Gc.edges())
nodes_dict = [{"id":n,
               "color":color[n],
              "title": displaylabel[n],
              "displaylabel":n,
              "degree":size[n],
              "x":pos[n][0]*1000,
              "y":pos[n][1]*1000} for n in nodes
              ]
node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

edges_dict = [{"source":node_map[edges[i][0]], 
               "target":node_map[edges[i][1]], 
              "color":"gray"
              } for i in range(len(edges))]

# set some network-wide styles
NodeGraphInfluencer = visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                          node_label_field='displaylabel',
                          node_size_multiplier=10,
                          node_color_highlight_border='red',
                          node_color_highlight_background='#D3918B',
                          node_color_hover_border='blue',
                          node_color_hover_background='#8BADD3',
                          node_font_size=25,
                          edge_arrow_to=False,
                          physics_enabled=True,
                          edge_color_highlight='#8A324E',
                          edge_color_hover='#8BADD3',
                          edge_label_field='label',
                          edge_title_field='title',
                          edge_width=1,
                          max_velocity=15,
                          min_velocity=1,
                          graph_id = 3)

NodeGraphInfluencer

In [111]:
###lets try the stream thing
!python twitterpackage/Chap02_03/twitter_streaming.py \#maga

^C
Exception ignored in: <module 'threading' from '/usr/lib/python3.5/threading.py'>
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 1288, in _shutdown
    t.join()
  File "/usr/lib/python3.5/threading.py", line 1054, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.5/threading.py", line 1070, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt


In [114]:
!python twitterpackage/Chap02_03/twitter_conversation.py stream__maga.jsonl

Name: 
Type: DiGraph
Number of nodes: 1349
Number of edges: 0
Average in degree:   0.0000
Average out degree:   0.0000
Traceback (most recent call last):
  File "twitterpackage/Chap02_03/twitter_conversation.py", line 32, in <module>
    sorted_replied = sorted(graph.degree_iter(), key=itemgetter(1), reverse=True)
AttributeError: 'DiGraph' object has no attribute 'degree_iter'


In [122]:
# Chap02-03/twitter_conversation.py
import sys
import json
from operator import itemgetter
import networkx as nx

fname = 'stream__maga.jsonl'
with open(fname) as f:
    graph = nx.DiGraph()
    for line in f:
        tweet = json.loads(line)
        if 'id' in tweet:
            graph.add_node(tweet['id'],
                           tweet=tweet['text'],
                           author=tweet['user']['screen_name'],
                           created_at=tweet['created_at'])
            if tweet['in_reply_to_status_id']:
                reply_to = tweet['in_reply_to_status_id']
                if tweet['in_reply_to_status_id'] in graph \
                and tweet['user']['screen_name'] != graph.node[reply_to]['author']:
                    graph.add_edge(tweet['in_reply_to_status_id'], tweet['id'])
    print(nx.info(graph))

    #sorted_replied = sorted(graph.degree_iter(), key=itemgetter(1), reverse=True)
    #most_replied_id, replies = sorted_replied[0]
    #print("Most replied tweet ({} replies):".format(replies))
    #print(graph.node[most_replied_id])

    #print("Longest discussion:")
    #longest_path = nx.dag_longest_path(graph)
    #for tweet_id in longest_path:
    #    node = graph.node[tweet_id]
    #    print("{} (by {} at {})".format(node['tweet'],
    #                                    node['author'],
    #                                    node['created_at']))

Name: 
Type: DiGraph
Number of nodes: 2780
Number of edges: 1
Average in degree:   0.0004
Average out degree:   0.0004


In [227]:
# Chap02-03/twitter_conversation.py
import sys
import json
from operator import itemgetter
import networkx as nx

fname = 'stream__maga.jsonl'
topic = fname.replace('stream__','')
topic = topic.replace('.jsonl','')
IDtoName = dict()
with open(fname) as f:
    graph = nx.DiGraph()
    for line in f:
        tweet = json.loads(line)
        graph.add_edge(topic,tweet['user']['id'])
        IDtoName[tweet['user']['id']] = tweet['user']['screen_name']
    print(nx.info(graph))

Name: 
Type: DiGraph
Number of nodes: 2110
Number of edges: 2109
Average in degree:   0.9995
Average out degree:   0.9995


In [228]:
IDtoName

{793887941709991936: 'bfhistory12',
 869976631280099329: 'PauWitkowski',
 962805816461053954: 'Satchalistic1',
 953031493978329091: 'JacobDavid27',
 707219381923635204: 'texastrumper',
 883461342358900741: 'BatzelKathy',
 938438348347146240: 'RalphMore5',
 722150503371710465: 'joeyd541',
 904711013525856264: 'TheBigNFLTuna',
 721807999073984521: 'cheryl_cds',
 772628773728509952: 'lizBeth_Hineni',
 969021473225523202: 'TeeGee69229434',
 702773194713772033: 'franklinstrauss',
 935978788588056577: 'NoMoreVan',
 747467499944448003: 'PeaceisMust777',
 19862796: 'BeautybyYB',
 369184791: 'iloveseth1234',
 2890268676: 'trujillo_vickie',
 334389274: 'Tea_Party_Chris',
 14454814: 'gillianbdoll',
 922924610600951813: 'amazingfavors',
 971316477126115329: 'Pootz4',
 950383610518847494: 'BlessedmessLady',
 4422399434: 'ValorOf1776',
 310329386: 'cpag001',
 818571375107514375: 'linda033159',
 239497261: 'Sedibus_MCML',
 149491758: 'EdwardCalame',
 923278397085835272: 'bluepillsheep',
 36048950: 'n

In [200]:
# Chap02-03/twitter_get_user.py
import os
import sys
import json
import time
import math
from tweepy import Cursor
from twitter_client import get_twitter_client

MAX_FRIENDS = 25000

def paginate(items, n):
    """Generate n-sized chunks from items"""
    for i in range(0, len(items), n):
        yield items[i:i+n]

screen_name = v
client = get_twitter_client()
max_pages = math.ceil(MAX_FRIENDS / 5000)

# get followers for a given user
fulllist = list()
for followers in Cursor(client.followers_ids, screen_name=screen_name).pages(max_pages):
    fulllist.append(followers)
flatlist = [item for sublist in fulllist for item in sublist]




In [245]:
v

'joyreaper'

In [250]:
# Chap02-03/twitter_get_user.py
import os
import sys
import json
import time
import math
from tweepy import Cursor
from twitter_client import get_twitter_client

MAX_FRIENDS = 25000

def paginate(items, n):
    """Generate n-sized chunks from items"""
    for i in range(0, len(items), n):
        yield items[i:i+n]

screen_name = v
client = get_twitter_client()
max_pages = math.ceil(MAX_FRIENDS / 5000)


# get followers for a given user
fulllist = list()
for followers in Cursor(client.followers_ids, screen_name=screen_name).pages(max_pages):
    fulllist.append(followers)
    status = client.rate_limit_status()
    nfollowers = status['resources']['followers']['/followers/ids']['remaining']
    while nfollowers <= 1:
            print("Sleeping for 5 seconds to check rate limit which is at {}".format(nfollowers))
            time.sleep(5)
            status = client.rate_limit_status()
            nfollowers = status['resources']['followers']['/followers/ids']['remaining']
            print("Checked API and limit is at {}".format(nfollowers))
flatlist = [item for sublist in fulllist for item in sublist]



Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit 

Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1
Checked API and limit is at 1
Sleeping for 5 seconds to check rate limit which is at 1


KeyboardInterrupt: 

[832028242852855808,
 982077025061167104,
 982208822985945088,
 954399180209913856,
 2324127125,
 307043670,
 19790638,
 713606252647985152,
 790920343439499269,
 815725918991708161,
 35800395,
 1729517072,
 960400921388662785,
 763394245008982016,
 888044919360593924,
 851920411377909760,
 1729010148,
 982638382282047488,
 932646714414116864,
 2344290223,
 902014202658873350,
 700717089548439552,
 974725666124873728,
 845141246238609409,
 986846336443207680,
 3575459537,
 225188002,
 966918077093851136,
 801648599784427520,
 856408498208260096,
 189940236,
 844236642,
 960907606452613120,
 4040155154,
 873680175191310336,
 2369245394,
 341183285,
 887529387188338688,
 818612851392679938,
 734987852820402176,
 4786132700,
 824074842311827457,
 878322668621004800,
 912273309366210561,
 932206667349012480,
 385678390,
 822235967591481344,
 892469822575050754,
 757776865180483584,
 47395667,
 715020643,
 233025830,
 21845804,
 24057319,
 882648784622235650,
 2319321097,
 98304759396068556

In [130]:
# Chap02-03/twitter_get_user.py
import os
import sys
import json
import time
import math
from tweepy import Cursor
from twitter_client import get_twitter_client

MAX_FRIENDS = 25000

def paginate(items, n):
    """Generate n-sized chunks from items"""
    for i in range(0, len(items), n):
        yield items[i:i+n]

screen_name = v
client = get_twitter_client()
dirname = "users/{}".format(screen_name)
max_pages = math.ceil(MAX_FRIENDS / 5000)
try:
    os.makedirs(dirname, mode=0o755)
except OSError:
    print("Directory {} already exists".format(dirname))
except Exception as e:
    print("Error while creating directory {}".format(dirname))
    print(e)
    sys.exit(1)

# get followers for a given user
fname = "users/{}/followers.jsonl".format(screen_name)
with open(fname, 'w') as f:
    for followers in Cursor(client.followers_ids, screen_name=screen_name).pages(max_pages):
        for chunk in paginate(followers, 100):
            users = client.lookup_users(user_ids=chunk)
            for user in users:
                f.write(json.dumps(user._json)+"\n")
        if len(followers) == 5000:
            print("More results available. Sleeping for 60 seconds to avoid rate limit")
            time.sleep(60)

# get friends for a given user
fname = "users/{}/friends.jsonl".format(screen_name)
with open(fname, 'w') as f:
    for friends in Cursor(client.friends_ids, screen_name=screen_name).pages(max_pages):
        for chunk in paginate(friends, 100):
            users = client.lookup_users(user_ids=chunk)
            for user in users:
                f.write(json.dumps(user._json)+"\n")
        if len(friends) == 5000:
            print("More results available. Sleeping for 60 seconds to avoid rate limit")
            time.sleep(60)

# get user's profile
fname = "users/{}/user_profile.json".format(screen_name)
with open(fname, 'w') as f:
    profile = client.get_user(screen_name=screen_name)
    f.write(json.dumps(profile._json, indent=4))

More results available. Sleeping for 60 seconds to avoid rate limit


KeyboardInterrupt: 

In [242]:
###get profile page
import sys
import os
import importlib
sys.path.insert(0,"twitterpackage/Chap02_03")
#print(sys.path)
#print(os.environ)

from twitter_client import get_twitter_client
import json
client = get_twitter_client()
profile = client.get_user(screen_name=v)
print(json.dumps(profile._json, indent =4))

{
    "profile_use_background_image": true,
    "entities": {
        "url": {
            "urls": [
                {
                    "expanded_url": "https://gab.ai/joyreaper",
                    "url": "https://t.co/R6fWEzNVSe",
                    "display_url": "gab.ai/joyreaper",
                    "indices": [
                        0,
                        23
                    ]
                }
            ]
        },
        "description": {
            "urls": []
        }
    },
    "profile_background_color": "185E50",
    "profile_image_url": "http://pbs.twimg.com/profile_images/966731780555509760/T5lhhNKb_normal.jpg",
    "following": false,
    "default_profile_image": false,
    "utc_offset": -18000,
    "has_extended_profile": true,
    "protected": false,
    "id_str": "28273116",
    "profile_sidebar_fill_color": "BCD6E0",
    "contributors_enabled": false,
    "listed_count": 336,
    "name": "Deplorable Trump Pilot Cosmonaut \ud83d\udd25",
    "id": 2